<a href="https://colab.research.google.com/github/searchf31/colab/blob/main/Torrent_To_Google_Drive_Downloader_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [13]:
# Install the system package
!pip install libtorrent

import sys
# Add the directory where apt installed the module
sys.path.append('/usr/lib/python3/dist-packages')

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
print("Libtorrent imported successfully!")

Libtorrent imported successfully!


/tmp/ipython-input-2721637194.py:11: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


### Mount Google Drive
To stream files we need to mount Google Drive.

In [14]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [15]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Saving FTUApps.com-Internet-Download-Manager-IDM-6.42-Build-56-Final-Multilingual-SUPER-CLEAN-Crack.torrent to FTUApps.com-Internet-Download-Manager-IDM-6.42-Build-56-Final-Multilingual-SUPER-CLEAN-Crack (1).torrent


### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [9]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


KeyboardInterrupt: Interrupted by user

### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [18]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
